### Starting AWS instance and running cifar10

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

#### Define parameters

In [3]:
vpc_name='fast-ai'

#### Get Existing VPC by tag name

In [4]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [5]:
instance_name = f'{vpc_name}-instance'
# instance_type = 't1.micro'
instance_type = 't2.small'
# instance_type = 'g2.2xlarge'

In [6]:
spot_price = get_spot_prices()[instance_type]
bid_price = "%.4f" % (float(spot_price)*3)
print(f'Spot price: {spot_price}, Bid price: {bid_price}')

Spot price: 0.015900, Bid price: 0.0477


In [7]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [8]:
launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 80

In [9]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 80,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-8c4288f4',
 'InstanceType': 't2.small',
 'KeyName': 'aws-key-fast-ai',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-f60fca88'],
   'SubnetId': 'subnet-f056ff89'}]}

In [10]:
instance = create_spot_instance(instance_name, launch_specs, spot_price=bid_price); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0299171c89b7e0d34
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@54.245.186.137


ec2.Instance(id='i-0299171c89b7e0d34')

Request on demand instance (if spot error)

In [ ]:
if not instance:
    instance = create_instance(instance_name, vpc, instance_type=instance_type); instance

In [ ]:
if not instance:
    instance = get_instance(instance_name); instance

### SSH

In [24]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EFS

In [12]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [13]:
run_command(client, 'mkdir ~/efs_mount_point')

('', '')

In [14]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [15]:
run_command(client, efs_mount_cmd)

('', '')

In [16]:
run_command(client, 'ls efs_mount_point') # no reformatting

('cifar10.tgz  efs_saved.txt\r\n', '')

### Upload cifar scripts

In [17]:
out, _ = run_command(client, 'mkdir cifar10')

In [29]:
upload_path = Path.cwd()/'upload_scripts/run_cifar10.sh'
upload_file(client, str(upload_path), 'cifar10/run_cifar10.sh')

('', '')

In [30]:
upload_path = Path.cwd()/'upload_scripts/cifar10.py'
upload_file(client, str(upload_path), 'cifar10/cifar10.py')

('', '')

## TMUX

In [26]:
tsess = TmuxSession(client, 'sess')

In [27]:
tsess.run_cmd('cd cifar10')

('', '')

In [33]:
tsess.run_cmd('bash run_cifar10.sh')

('', '')

# Testing
